In [1]:
from src.models.testing_model import model_builder_v0
from config import GLOBAL_CONFIG_DATA_PATH, GLOBAL_CONFIG_DATA_RAW_IMAGE_PATH, GLOBAL_CONFIG_DATA_RAW_LABEL_PATH, GLOBAL_CONFIG_RESULT_PATH

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Load the data from CSV
csv_file = GLOBAL_CONFIG_DATA_RAW_LABEL_PATH  # Replace with your CSV file path
data = pd.read_csv(csv_file)

# Print the first few rows of the dataframe
print(data.head())

# Prepare the image data
def load_and_preprocess_image(image_id):
    # Load the image using the image ID (assuming images are stored in a specific directory)
    img_path = f'{GLOBAL_CONFIG_DATA_RAW_IMAGE_PATH}{image_id}'  # Modify the path and extension as needed
    img = load_img(img_path, target_size=(30, 30), color_mode='grayscale')  # Adjust size if needed
    img_array = img_to_array(img) / 255.0  # Normalize the image
    return img_array

# Create arrays for images and labels
images = []
labels = []

for index, row in data.iterrows():
    img = load_and_preprocess_image(row['Image_ID'])
    images.append(img)
    labels.append(row['class'])  # Use the 'class' column for labels

images = np.array(images)
labels = np.array(labels)


            Image_ID        class  confidence  ymin  xmin  ymax  xmax
0  id_u3q6jdck4j.jpg  Trophozoite         1.0   712  1241   737  1270
1  id_a6cl90trri.jpg  Trophozoite         1.0   558  1566   600  1604
2  id_qvc2le9sm8.jpg  Trophozoite         1.0  1317  2788  1448  2914
3  id_w8xnbd5rvm.jpg  Trophozoite         1.0   925  1744  1041  1823
4  id_6dop09rk02.jpg          NEG         1.0     0     0     0     0


ValueError: invalid literal for int() with base 10: 'Trophozoite'

In [11]:
labels

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [17]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [29]:
# Define the input shape
input_shape = (30, 30, 1)
inputs = layers.Input(shape=input_shape)

# Branch 1: l1a
l1a = layers.Conv2D(5, (5, 5), strides=(1, 1), activation='relu')(inputs)

# Branch 2: l1b
l1b = layers.Conv2D(7, (3, 3), strides=(1, 1), activation='relu')(inputs)
l1b2 = layers.Conv2D(1, (1, 1), strides=(1, 1), activation='relu')(l1b)  # Compress to 1 channel

# From l1b2 to l2a
l2a = layers.Conv2D(5, (3, 3), strides=(1, 1), activation='relu')(l1b2)

# Max pooling from l2a to l2a2
l2a2 = layers.MaxPooling2D((2, 2), strides=(2, 2))(l2a)

# Dense layer from l2a to l3a
l3a = layers.Flatten()(l2a2)
l3a = layers.Dense(5, activation='relu')(l3a)

# max pooling from l1a to l1a2
l1a2 = layers.MaxPooling2D((2, 2), strides=(2, 2))(l1a)

# Flatten l1a to l2b
l2b = layers.Flatten()(l1a2)

# Combine l3a and l2b into l4a
l4a = layers.concatenate([l3a, l2b])

# Final output layer (example: for binary classification)
output = layers.Dense(3, activation='softmax')(l4a)

# Create the model
model = models.Model(inputs=inputs, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'f1_score'])

model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, 30, 30, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_17 (Conv2D)  │ (None, 28, 28, 7) │         70 │ input_layer_7[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_18 (Conv2D)  │ (None, 28, 28, 1) │          8 │ conv2d_17[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_19 (Conv2D)  │ (None, 26, 26, 5) │         50 │ conv2d_18[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_8     │ (None, 13, 13, 5) │          0 │ conv2d_19[0][0]   │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_16 (Conv2D)  │ (None, 26, 26, 5) │        130 │ input_layer_7[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_8 (Flatten) │ (None, 845)       │          0 │ max_pooling2d_8[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_9     │ (None, 13, 13, 5) │          0 │ conv2d_16[0][0]   │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 5)         │      4,230 │ flatten_8[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_9 (Flatten) │ (None, 845)       │          0 │ max_pooling2d_9[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 850)       │          0 │ dense_8[0][0],    │
│ (Concatenate)       │                   │            │ flatten_9[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 3)         │      2,553 │ concatenate_4[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,041 (27.50 KB)

 Trainable params: 7,041 (27.50 KB)

 Non-trainable params: 0 (0.00 B)

In [30]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Build the model (assuming you have the build_alexnet_with_residuals function defined)
# model_builder = model_builder_v0()

# input_shape = (30, 30, 1)

# model = model_builder.create_model(output_shape=3)  # Adjust input shape and num_classes as needed

# Train the model
history = model.fit(X_train, y_train, 
                    validation_data=(X_val, y_val),
                    batch_size=32, 
                    epochs=20)  # Adjust epochs as necessary

# Save the model
model.save(f'{GLOBAL_CONFIG_RESULT_PATH}testing/mv0/model1.h5')  # Save the trained model for later use

# Optional: Print training history
print(history.history)

Epoch 1/20
589/589 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6522 - f1_score: 0.3910 - loss: 0.7202 - val_accuracy: 0.7187 - val_f1_score: 0.7412 - val_loss: 0.5689
Epoch 2/20
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7227 - f1_score: 0.7188 - loss: 0.5528 - val_accuracy: 0.7346 - val_f1_score: 0.7286 - val_loss: 0.5367
Epoch 3/20
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7312 - f1_score: 0.7372 - loss: 0.5330 - val_accuracy: 0.7337 - val_f1_score: 0.7212 - val_loss: 0.5361
Epoch 4/20
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7412 - f1_score: 0.7501 - loss: 0.5224 - val_accuracy: 0.7310 - val_f1_score: 0.7616 - val_loss: 0.5356
Epoch 5/20
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7402 - f1_score: 0.7519 - loss: 0.5180 - val_accuracy: 0.7395 - val_f1_score: 0.7567 - val_loss: 0.5228
Epoch 6/20
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7501 - f1_score: 0.7655 - loss: 0.5074 - val_accuracy: 0.7465 - val_f1_score: 0.7629

{'accuracy': [0.6868890523910522, 0.7216851115226746, 0.729547381401062, 0.7366659641265869, 0.7413408160209656, 0.7483000159263611, 0.7520717978477478, 0.7547280192375183, 0.7578623294830322, 0.7625371813774109, 0.7640246748924255, 0.7621122002601624, 0.765352725982666, 0.7655121088027954, 0.7699213624000549, 0.7699213624000549, 0.7669996023178101, 0.7701338529586792, 0.771568238735199, 0.7717807292938232], 'f1_score': [<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.65506643, 0.80583096, 0.15215091], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.9991079 , 0.81599087, 0.3500806 ], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([1.        , 0.81934625, 0.38993403], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([1.       , 0.8230337, 0.4178508], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([1.        , 0.82562757, 0.4340346 ], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=arr